In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

llm = ChatOpenAI()

In [3]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)

{'topic': 'pizza',
 'joke': 'Why did the pizza go to the doctor? Because it had too many toppings and was feeling saucy!',
 'explanation': 'This joke plays on the idea of a pizza needing to see a doctor because it has "too many toppings" and is "feeling saucy." In reality, pizzas are inanimate objects and cannot feel anything or go to the doctor. The joke is meant to be funny because it personifies the pizza and gives it human-like qualities, which is unexpected and silly. The play on words with "feeling saucy" also adds humor to the punchline.'}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it had too many toppings and was feeling saucy!', 'explanation': 'This joke plays on the idea of a pizza needing to see a doctor because it has "too many toppings" and is "feeling saucy." In reality, pizzas are inanimate objects and cannot feel anything or go to the doctor. The joke is meant to be funny because it personifies the pizza and gives it human-like qualities, which is unexpected and silly. The play on words with "feeling saucy" also adds humor to the punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-681d-6133-8002-48bbc6b15038'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T13:45:47.983697+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-578a-6661-8001-c22f9e1bcb57'}}, tasks=(), interrupts=())

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it had too many toppings and was feeling saucy!', 'explanation': 'This joke plays on the idea of a pizza needing to see a doctor because it has "too many toppings" and is "feeling saucy." In reality, pizzas are inanimate objects and cannot feel anything or go to the doctor. The joke is meant to be funny because it personifies the pizza and gives it human-like qualities, which is unexpected and silly. The play on words with "feeling saucy" also adds humor to the punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-681d-6133-8002-48bbc6b15038'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T13:45:47.983697+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-578a-6661-8001-c22f9e1bcb57'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'piz

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the pasta chef go to therapy? \nBecause he had too much emotional baggage-gini!',
 'explanation': 'This joke plays on the double meaning of "baggage." In the context of pasta, "baggage" refers to the type of noodle, while in the context of therapy, "baggage" refers to emotional issues or trauma. The joke suggests that the pasta chef went to therapy because he had too much emotional "baggage-gini," a play on the word "fettuccini." This humorous twist on the familiar term "emotional baggage" adds a lighthearted and punny element to the joke.'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza go to the doctor? Because it had too many toppings and was feeling saucy!', 'explanation': 'This joke plays on the idea of a pizza needing to see a doctor because it has "too many toppings" and is "feeling saucy." In reality, pizzas are inanimate objects and cannot feel anything or go to the doctor. The joke is meant to be funny because it personifies the pizza and gives it human-like qualities, which is unexpected and silly. The play on words with "feeling saucy" also adds humor to the punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-681d-6133-8002-48bbc6b15038'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T13:45:47.983697+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f089957-578a-6661-8001-c22f9e1bcb57'}}, tasks=(), interrupts=())

In [17]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the pasta chef go to therapy? \nBecause he had too much emotional baggage-gini!', 'explanation': 'This joke plays on the double meaning of "baggage." In the context of pasta, "baggage" refers to the type of noodle, while in the context of therapy, "baggage" refers to emotional issues or trauma. The joke suggests that the pasta chef went to therapy because he had too much emotional "baggage-gini," a play on the word "fettuccini." This humorous twist on the familiar term "emotional baggage" adds a lighthearted and punny element to the joke.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f089965-059b-6a36-8002-8035d5f8e767'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T13:51:53.464274+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f089964-f2bc-6515-8001-47fdd6425959'}}, tasks=(), interrupts=()),
 Sta

# Time Travel

In [21]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f089964-e152-6656-8000-f356a4e2d38d"}})

StateSnapshot(values={'topic': 'pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f089964-e152-6656-8000-f356a4e2d38d'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-04T13:51:49.659400+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f089964-e14f-6e9b-bfff-20482407f416'}}, tasks=(PregelTask(id='e88fc63f-4476-aa75-853a-9fea3551a2bd', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pasta chef go to therapy? \nBecause he had too much emotional baggage-gini!'}),), interrupts=())

In [22]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f089964-e152-6656-8000-f356a4e2d38d"}})

{'topic': 'pasta',
 'joke': 'Why did the spaghetti break up with the linguine? Because it heard she was alfredo-ing around!',
 'explanation': 'This joke plays on the similarity between the word "alfredo" and the phrase "all for doing." The joke suggests that the spaghetti broke up with the linguine because it heard she was engaging in promiscuous behavior or "alfredo-ing around," which sounds like the phrase "all for doing around." It\'s a pun-based joke that relies on wordplay for its humor.'}

In [23]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the linguine? Because it heard she was alfredo-ing around!', 'explanation': 'This joke plays on the similarity between the word "alfredo" and the phrase "all for doing." The joke suggests that the spaghetti broke up with the linguine because it heard she was engaging in promiscuous behavior or "alfredo-ing around," which sounds like the phrase "all for doing around." It\'s a pun-based joke that relies on wordplay for its humor.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f08999a-21c1-666f-8002-a0f036768297'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-04T14:15:39.123671+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f08999a-0678-6628-8001-716fee548f60'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the

# Updating State

In [24]:
workflow.update_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f089964-e152-6656-8000-f356a4e2d38d", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '2',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0899a0-75de-6862-8001-38c59511d2f7'}}

In [26]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0899a0-75de-6862-8001-38c59511d2f7'}}, metadata={'source': 'update', 'step': 1, 'parents': {}}, created_at='2025-09-04T14:18:29.004912+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f089964-e152-6656-8000-f356a4e2d38d'}}, tasks=(PregelTask(id='053a1c27-0341-1c4c-8a0c-c0d39caab70c', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti break up with the linguine? Because it heard she was alfredo-ing around!', 'explanation': 'This joke plays on the similarity between the word "alfredo" and the phrase "all for doing." The joke suggests that the spaghetti broke up with the linguine because it heard she was engaging in promiscuous behavior or

In [28]:
workflow.invoke(None, {"configurable": {"thread_id": "2", "checkpoint_id": "1f0899a0-75de-6862-8001-38c59511d2f7"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?\nBecause it had too many layers to unravel!',
 'explanation': 'This joke plays on the idea of the samosa, a popular Indian snack, going to therapy like a person would. The punchline, "Because it had too many layers to unravel," combines the literal layers of a samosa (made with multiple layers of dough and filling) with the metaphorical idea of someone having emotional or psychological layers that need to be unpacked in therapy. The humor comes from the unexpected comparison and the word play on "layers."'}

In [29]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?\nBecause it had too many layers to unravel!', 'explanation': 'This joke plays on the idea of the samosa, a popular Indian snack, going to therapy like a person would. The punchline, "Because it had too many layers to unravel," combines the literal layers of a samosa (made with multiple layers of dough and filling) with the metaphorical idea of someone having emotional or psychological layers that need to be unpacked in therapy. The humor comes from the unexpected comparison and the word play on "layers."'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0899ab-a7fa-67c2-8003-ddbcba6904fa'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-04T14:23:29.538246+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0899ab-91cb-6c43-8002-365398158d0e'}}, tasks=(), interrupts=()),
 StateSnapsh

# Fault Tolerance

In [30]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

In [31]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [32]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [33]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)


In [1]:
# 6. Re-run to show fault-tolerant resume
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...


NameError: name 'graph' is not defined

In [ ]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))